In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('C:/Users/연구실/연구실 프로젝트/data/전처리최종이었음좋겠다.csv')
data.head()

,yearmonth,Ticker,한국표준산업분류코드10차(중분류),수익률 (1개월)(%),"베타 (D,1Yr)",PER(배),PSR(배),chmom,mom36m,zerotrade,...,원/달러환율,금($/OZ),뉴스심리지수,KOSPI dp,KOSPI PER,KOSPI PBR,CD Rate,Term spread,Default Spread,VKOSPI 200
0,2004-01,A005930,26,1.87,0.756757,-0.133333,0.670270,-0.084685,0.812613,-1.0,...,1172.1,402.45,100.0,1.99,10.72,1.07,4.26,1.16,4.32,21.23
1,2004-02,A005930,26,4.95,0.764388,-0.124101,0.688849,-0.383094,0.879496,-1.0,...,1171.6,396.15,100.0,1.99,10.94,1.08,4.08,1.35,4.40,19.95
2,2004-03,A005930,26,-0.54,0.861261,-0.072072,0.713514,0.623423,0.846847,-1.0,...,1153.3,426.45,100.0,1.83,11.19,1.11,3.93,1.33,4.48,24.66
3,2004-04,A005930,26,-9.16,0.905626,-0.081670,0.709619,-0.618875,0.809437,-1.0,...,1170.7,386.75,100.0,2.21,16.76,1.10,3.91,1.34,4.47,24.47
4,2004-05,A005930,26,-8.27,0.881081,-0.079279,0.720721,-0.621622,0.881081,-1.0,...,1164.9,395.55,100.0,2.39,15.40,1.02,3.90,1.26,4.47,30.14


In [ ]:
X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', '뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [ ]:
# cross_validate 적용
import warnings
warnings.filterwarnings("ignore")

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm
import joblib

a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
sse = 0
ya0 = []
tk0 = []
ym0 = []
ya1 = []

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvym = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    Vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    gbrt_search_space = {
        'learning_rate' : hp.choice('learning_rate', [0.00001, 0.0001, 0.001, 0.01, 0.1]),
        'max_depth' : hp.quniform('max_depth', 1, 6, 1),
        'n_estimators' : hp.quniform('n_estimators', 100, 500, 10),
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        gbrt = GradientBoostingRegressor(
            loss='huber', 
            alpha=0.999, 
            n_estimators = int(search_space['n_estimators']),
            learning_rate = search_space['learning_rate'],
            max_depth = int(search_space['max_depth']),
            random_state = 1
        )
        
        d = 0
        
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            gbrt.fit(tX, ty)
            
            # r2oos 계산         
            y_pred = pd.Series(gbrt.predict(vX).reshape(-1,)) # sort_values() 사용을 위해 변환
            d = d + sum((vy-y_pred)**2) 

        e = sum(Vy**2)
        r2oos = 1-(d/e)
        new_loss = r2oos*(-1)

        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=gbrt_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        return_argmin=False, # hp.choice의 경우 값 대신 index를 반환함 
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['n_estimators'] = int(best['n_estimators'])
    best['max_depth'] = int(best['max_depth'])
    print(best) 
    
    reg0 = GradientBoostingRegressor(
        loss = 'huber', 
        alpha = 0.999,
        random_state = 1,
        **best)
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    joblib.dump(reg0,'GBRT_bayesian_r2oos_last3_'+str(i)+'.pkl')
    sse = sse + sum((oy-yy0)**2)

oyy = y[ym.str.contains(a[40:84])]
e = sum(oyy**2)
r2oos = 1-(sse/e)
print(r2oos)
    
GBRT = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
GBRT.to_csv('GBRT_bayesian_r2oos.csv')    

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]


  3%|█                                  | 31/1000 [9:37:34<300:54:07, 1117.90s/trial, best loss: -0.022526003591763266]
{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 110}


 11%|████████▍                                                                   | 1/9 [9:38:56<77:11:31, 34736.46s/it]


  3%|█                                 | 31/1000 [17:14:29<538:56:07, 2002.24s/trial, best loss: -0.004626429003620425]
{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 110}


 22%|████████████████▋                                                          | 2/9 [26:55:06<98:55:16, 50873.81s/it]

...


  2%|▋                                | 20/1000 [21:30:22<1053:48:23, 3871.13s/trial, best loss: -0.011047500530663279]
{'learning_rate': 0.001, 'max_depth': 4, 'n_estimators': 430}


 89%|█████████████████████████████████████████████████████████████████▊        | 8/9 [119:15:54<16:59:27, 61167.69s/it]


  2%|▋                                | 20/1000 [23:13:06<1137:42:31, 4179.34s/trial, best loss: -0.008877642287229759]
{'learning_rate': 0.001, 'max_depth': 4, 'n_estimators': 430}


100%|█████████████████████████████████████████████████████████████████████████████| 9/9 [143:12:04<00:00, 57280.47s/it]


-0.003120370577061937


In [ ]:
import pandas as pd
import numpy as np
GBRT = pd.read_csv("GBRT_bayesian_r2oos.csv")

In [ ]:
ya0 = pd.Series(GBRT['real'])
ya1 = pd.Series(GBRT['predict'])
tk0 = pd.Series(GBRT['ticker'])
ym0 = pd.Series(GBRT['yearmonth'])

r1 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())

rr1 = 1
rgbrt1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rgbrt1.append(rr1)

In [ ]:
rgbrt1[-1]

27.77815124446899

In [ ]:
r1 = []
r2 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

In [ ]:
for i in range(9):
    rr1 = 1
    for j in r1[0+i*12:12+i*12]:
        rr1 = rr1*(j+100)/100
    print("year:" ,i, "test 수익률:", rr1)

year: 0 test 수익률: 1.5743718514920493
year: 1 test 수익률: 1.5234573193313339
year: 2 test 수익률: 1.7142262378324884
year: 3 test 수익률: 1.519993217272505
year: 4 test 수익률: 1.8580196299859524
year: 5 test 수익률: 1.197174590519485
year: 6 test 수익률: 1.1391020443282107
year: 7 test 수익률: 1.1562081251344234
year: 8 test 수익률: 1.51722330598476
